# Поиск токсичных комментариев

#### Тема проекта:
- Определение токсичных комментариев пользователей интернет-магазина

#### Цель:
- Обучить модель классифицировать комментарии на позитивные и негативные (токсичные)

#### Поставленные задачи:
- Исследовать предоставленные данные;
- Использовать несколько способов подготовки текстовых данных для обучения;
- Обучить несколько моделей и сравнить результаты;
- Получить значение F1-меры не ниже 0.75.

#### Краткий план работы:
- [Шаг 1. Подготовка данных](#Шаг-1.-Подготовка-данных)
- [Шаг 2. Преобразование текстовых данных](#Шаг-2.-Преобразование-текстовых-данных)
- [Шаг 3. Обучение моделей](#Шаг-3.-Обучение-моделей)
  - [3.1. Логистическая регрессия (LogisticRegression)](#3.1.-Логистическая-регрессия-(LogisticRegression))
  - [3.2. Стохастический градиентный спуск (SGDClassifier)](#3.2.-Стохастический-градиентный-спуск-(SGDClassifier))
  - [3.3. Градиентный бустинг (LGBMClassifier)](#3.3.-Градиентный-бустинг-(LGBMClassifier))
  
#### Вывод:
- Исследованы предоставленные данные;
- Использовано три способа подготовки текстовых данных для обучения;
- На тестовой выборке удалось достигнуть значения F1-меры не ниже 0.75.

**Статус проекта**: проект завершён.

**Используемые библиотеки**: *numpy*, *pandas*, *re*, *tqdm*, *sklearn*, *lightgbm*, *textblob*, *gensim*, *nltk*

**Источник данных**: [курс Data Science от Яндекс.Практикум](https://praktikum.yandex.ru/profile/data-scientist/)

In [1]:
# отключение предупреждений
import warnings
warnings.filterwarnings('ignore') 

# импорт библиотек и функция для дальнейшей работы
import pandas as pd
import numpy as np
import re
from tqdm import notebook

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import f1_score, classification_report
from sklearn.linear_model import LogisticRegression, SGDClassifier
from lightgbm import LGBMClassifier

In [2]:
# работа с текстом
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob, Word

from gensim.test.utils import datapath
from gensim import utils
import gensim.models
import gensim.downloader as api

import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

# стоп-слова
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\marne\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marne\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marne\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Шаг 1. Подготовка данных

Выгрузим исходные данные, изучим общую информацию, выведем первые строки и проверим данные на наличие дубликатов.   
Дополнительно выведем **процент токсичный комментариев** в датасете.

In [3]:
# выгрузка файла
df = pd.read_csv('datasets/toxic_comments.csv')

print('Количество дубликатов:', len(df[df.duplicated()]))
print('Доля токсичных комментариев: {:.2%}'.format(df['toxic'].sum() / df['toxic'].count()))
print()
print(df.info())
df.head()

Количество дубликатов: 0
Доля токсичных комментариев: 10.17%

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
None


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


- Файл содержит **159571 комментариев**; 
- **Пропусков** и **дубликатов** не обнаружено;
- Около **10%** всех комментариев являются токсичными, т. е. наблюдается дисбаланс классов.

Создадим дополнительный столбец с **лемматизированным текстом**.  
Для этих целей используем **TextBlob Lemmatizer** с соответствующим POS-тегом.

Напишем функцию, к-я будет к-я будет лемматизировать текст, выбирая для каждого слова наиболее вероятную часть речи.

In [4]:
def lemmatize_with_postag(sentence):   
    """Функция возвращает лемматизированный текст, подбирая для каждого слова часть речи:
    - sentence - указать предложение для лемматизации."""
    
    # лемматизация предложения
    sent = TextBlob(sentence)
    
    # подбор части речи для нестандартных обозначений
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    
    # создание списка (слово, часть речи)
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
    
    # список с лемматизированным текстом
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

In [5]:
# проверка работы функции
sentence = """Like many other startup companies, Uber was founded on the premise of disruption:
taking an old industry, oftentimes one that was a bit clunky. For a short period of time, companies like Uber were
viewed as economic saviors. They sold themselves as a means of better using and distributing resources."""

lemmatize_with_postag(sentence)

'Like many other startup company Uber be found on the premise of disruption take an old industry oftentimes one that be a bit clunky For a short period of time company like Uber be view a economic savior They sell themselves a a mean of good use and distributing resource'

Лемматизатор не всегда выдаёт правильную форму, но в целом он понимает:
- Множественное число: *companies - company*
- Герундий: *taking - take*
- Прошедшее время: *sold - sell*
- Глагол *be* в разных формах: *were - be*
- Сравнительную форму прилагательных *good*, *bad*: *better* - *good*.

**Очистим** текст от ненужных символов и оставим только латинские буквы, использовав **шаблон**:

In [6]:
def clear_text(text):    
    """Функция убирает небуквенные символы и оставляет только слова, написанные латиницей:
    - text - указать текст для очистки."""   
    cleared = re.sub(r'[^a-zA-z]', ' ', text)
    return " ".join(cleared.split())

Напишем функцию для создания колонки с **обработанным текстом**:

In [7]:
def lemmatize_df(df_name, batch_size=1000):
    
    """Функция добавляет колонку с преобразованным текстом, обрабатывая исходный датафрейм по частям:
    - df_name - наименование датафрейма;
    - batch_size - размер батча (по умолчанию 1000 строк)"""
    
    # пустой финальный датафрейм
    df_final = pd.DataFrame()
    
    # цикл для обработки датафрейма по частям
    for i in notebook.tqdm(range(df_name.shape[0] // batch_size + 1)):
        
        df_part = df_name.iloc[batch_size*i : batch_size*(i+1)]
            
        # перевод всех символов в нижний регистр
        df_part['lemm_text'] = df_part['text'].apply(str.lower)
        
        # очистка текста от небуквенных символов
        df_part['lemm_text'] = df_part['lemm_text'].apply(clear_text)

        # лемматизация
        df_part['lemm_text'] = df_part['lemm_text'].apply(lemmatize_with_postag)
        
        df_final = df_final.append(df_part)
    
    return df_final

Создадим новый датафрейм с лемматизированным и очищенным от посторонних символов текстом.

In [8]:
# создание нового датафрейма
df_lemm = lemmatize_df(df)

print(df_lemm.info())
df_lemm.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   text       159571 non-null  object
 1   toxic      159571 non-null  int64 
 2   lemm_text  159571 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.7+ MB
None


,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


Дополнительно проверим столбец *lemm_text* на наличие пропусков:

In [9]:
df_lemm_nan = df_lemm[df_lemm['lemm_text'].isnull()]
print('Количество пропусков в lemm_text:', len(df_lemm_nan))
df_lemm_nan

Количество пропусков в lemm_text: 0


,text,toxic,lemm_text


В данных отсутствуют **пропуски**, можем приступать к следующему шагу.

Для дальнейших преобразований выделим **обучающую** и **тестовую** выборки в пропорции 60 на 40.

In [10]:
SEED = 18

# выделение обучающей и тестовой выборки
df_train = df_lemm.sample(frac=0.6, random_state=SEED)
df_test = df_lemm[~df_lemm.index.isin(df_train.index)]

# выделение признаков и целевого признака
X_train = df_train['lemm_text']
y_train = df_train['toxic']
X_test = df_test['lemm_text']
y_test = df_test['toxic']

print('Количество элементов в обучающей выборке:', X_train.shape[0])
print('Количество элементов в тестовой выборке:', X_test.shape[0])

Количество элементов в обучающей выборке: 95743
Количество элементов в тестовой выборке: 63828


## Шаг 1. Вывод

#### Изучен файл:
- Файл содержит **159571 комментариев**, из к-х **10%** являются **токсичными**.
- **Пропусков** и **дубликатов** не обнаружено.
    
#### Проведена лемматизация текста:
- Леммы выделены с помощью **TextBlob Lemmatizer**.
- Из-за большого объема исходный датафрейм был преобразован **по частям (батчам)**.
- Выделены **обучающая** и **тестовая** выборки в пропорции 60:40.

## Шаг 2. Преобразование текстовых данных

Для подготовки текстовых данных для дальнейшего обучения воспользуемся **несколькими техниками**:
- [2.1. Метод TF-IDF](#2.1.-Метод-TF-IDF)
- [2.2. Использование Word2Vec](#2.2.-Использование-Word2Vec)
- [2.3. Использование сторонних корпусов](#2.3.-Использование-сторонних-корпусов)

### 2.1. Метод TF-IDF

Проведём сентимент-анализ, использовав расчёт величины **TF-IDF** для каждого текста.

Для обучения необходимо привести **признаки** в соответствующий вид. Разделим выборку на **обучающую** и **тестовую** и для каждой из выборок преобразуем признаки, использовав `TfidfVectorizer`.

In [11]:
# создание TfidfVectorizer
count_tf_idf = TfidfVectorizer(stop_words = stopwords)

# выборка train
corpus_train = X_train.values.astype('U')
X_train_tfidf = count_tf_idf.fit_transform(corpus_train)
print("Размер матрицы train:", X_train_tfidf.shape)

# выборка test
corpus_test = X_test.values.astype('U')
X_test_tfidf = count_tf_idf.transform(corpus_test)
print("Размер матрицы test:", X_test_tfidf.shape)

Размер матрицы train: (95743, 117443)
Размер матрицы test: (63828, 117443)


Итак, мы перевели **признаки** в необходимый для обучения вид.

### 2.2. Использование Word2Vec

Представим наши слова в виде **векторов** с помощью метода построения языковых представлений **word2vec**.

In [12]:
class MyCorpus(object):
    """Итератор, который выдает обработанное предложение."""    
    def __iter__(self):                
        for line in X_train.values: # обучающая выборка
            # в строке одна запись, разделенная пробелами
            yield utils.simple_preprocess(line)

In [13]:
# создание образца класса
sentences = MyCorpus()
w2v = gensim.models.Word2Vec(sentences=sentences)

In [14]:
# проверка работы
print('Наиболее близкие слова:')
print(w2v.most_similar('discussion'))
print()

print('Векторное представление:')
print(w2v['discussion'])

Наиболее близкие слова:
[('debate', 0.7768642902374268), ('rfc', 0.621604859828949), ('conversation', 0.6086463332176208), ('thread', 0.6054226160049438), ('afd', 0.5890777707099915), ('comment', 0.5802565217018127), ('consensus', 0.5626112222671509), ('ani', 0.5513968467712402), ('discuss', 0.5490802526473999), ('dispute', 0.5428581237792969)]

Векторное представление:
[-3.0868955   0.42404908  2.7263534  -0.8208147  -2.148401    2.8722901
  1.2545977  -0.41676378  3.121291    1.4188262  -1.0123551  -0.4688601
 -2.9280999  -1.3276205   2.7306988  -2.4798932   3.6329334  -2.4391897
 -0.7627519  -0.27007627  1.5489968  -0.23699255 -0.44199198  0.37721848
  0.20171142  3.4347224  -2.8656614  -1.3479848  -0.80399007  0.32610008
 -1.3552976  -0.58394593  0.08817018 -0.16374959 -1.0531813   0.48285526
  0.2503096  -0.38590214 -1.8812141  -0.3798565   0.01637984 -1.9802777
 -3.2029028  -1.4443629   0.16734827 -0.49736717  2.03025    -3.0164073
  0.4867485  -2.9834347   1.7389363   0.1163712 

Для получения суммарных **векторов** по всей строке напишем функцию, которая будет принимать на вход строку и складывать в ней все вектора.

In [15]:
def get_vectors(row):
    """Функция принимает на вход строку и выдает сумму векторов."""   
    vecs = [np.zeros(100)] # вектор заполняется нулями
    
    for word in utils.simple_preprocess(row):
        try:
            vector = w2v[word]
        except:
            vector = np.zeros(100) # если новое слово, чтобы модель не выдала ошибку, сделаем заполнение нулями
        vecs.append(vector)
        
    return np.sum(np.array(vecs), axis=0)

In [16]:
# перевод 
X_train_vw = X_train.apply(get_vectors)
X_test_vw = X_test.apply(get_vectors)

X_train_vw = pd.DataFrame(X_train_vw.to_list())
X_test_vw = pd.DataFrame(X_test_vw.to_list())

print("Размер матрицы train:", X_train_vw.shape)
print("Размер матрицы test:", X_test_vw.shape)

Размер матрицы train: (95743, 100)
Размер матрицы test: (63828, 100)


In [17]:
# вывод нескольких строк
print('Векторные значения:')
X_test_vw.head()

Векторные значения:


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-19.143094,6.001502,4.879802,-20.931625,-17.077759,22.206276,5.614954,-15.418019,20.530026,14.200071,...,-1.811645,-28.734429,8.731041,21.691564,8.653783,10.486366,-6.300555,2.425375,7.174889,14.669803
1,-16.377851,9.115148,1.233155,-29.787657,-25.885999,16.485670,8.186777,-16.994646,47.848925,20.773394,...,3.052235,-42.483007,24.919971,26.989490,-12.588672,7.161145,3.161629,-2.554231,1.208275,2.393457
2,2.549082,4.965007,-1.214038,-5.276573,-14.215235,2.852126,1.038043,-6.409001,8.126548,4.270383,...,-3.514260,-7.959220,5.352983,4.553497,-6.374660,2.872086,-4.318406,-3.123333,-2.142425,1.448300
3,-5.825354,5.732467,8.188472,-13.569016,-22.181368,14.738446,8.062176,-10.192916,16.839786,10.515175,...,6.216455,-13.937288,3.456885,16.229399,5.464751,3.663197,-3.650932,2.354220,-4.489151,-1.423013
4,-47.559546,10.653450,-25.181754,-36.298261,-66.837556,33.488656,9.884977,-17.041061,65.342199,33.004103,...,8.437258,-78.356642,32.521686,57.653056,-11.413193,1.911562,-19.040217,-2.502533,-38.604903,7.508386


### 2.3. Использование сторонних корпусов

Для переведения текстовых данных в вектора мы можем использовать не только наши непосредственные данные, но и брать сторонние корпуса. В данном случае проведем обучение **на данных из твиттера**.

Источник данных: https://github.com/RaRe-Technologies/gensim-data

In [18]:
# загрузка модели из glove-twitter-50
gt50 = api.load('glove-twitter-50')

In [19]:
# проверка работы
print('Наиболее близкие слова:')
print(gt50.most_similar('discussion'))
print()

print('Векторное представление:')
print(gt50['discussion'])

Наиболее близкие слова:
[('article', 0.842178225517273), ('conversation', 0.7992870211601257), ('dialogue', 0.7992134094238281), ('discuss', 0.7935640215873718), ('discussions', 0.7880508899688721), ('summary', 0.7877680063247681), ('questions', 0.7793878316879272), ('articles', 0.7715831398963928), ('regarding', 0.7698727250099182), ('content', 0.7649972438812256)]

Векторное представление:
[ 1.457     0.34705  -0.070119  0.23195   1.5397    0.47888   0.57762
 -0.6027   -0.053207 -0.623    -0.39628  -0.71816  -2.5486    0.063757
 -0.11111   0.3769    0.3596    0.38589  -0.26696   0.19303  -0.36622
 -0.65895   0.038764 -0.3426   -0.4924    0.27062   0.29772  -0.060013
 -0.88368   0.65449   0.20583  -0.36151  -0.64415  -0.56266   0.41122
 -0.078038 -0.028325  0.45015   1.2699   -0.60587  -0.021684 -0.069277
  1.5698    0.35513   0.31453   0.22604  -0.21975   0.11178  -0.071925
 -0.60919 ]


In [20]:
def get_vectors_gt50(row):
    """Функция принимает на вход строку и выдает сумму векторов."""
    vecs = [np.zeros(50)]
    
    for word in utils.simple_preprocess(row):
        try:
            vector = gt50[word]
        except:
            vector = np.zeros(50)
        vecs.append(vector)
    return np.sum(np.array(vecs), axis=0)

In [21]:
# перевод 
X_train_gt50 = X_train.apply(get_vectors_gt50)
X_test_gt50 = X_test.apply(get_vectors_gt50)

X_train_gt50 = pd.DataFrame(X_train_gt50.to_list())
X_test_gt50 = pd.DataFrame(X_test_gt50.to_list())

print("Размер матрицы train:", X_train_gt50.shape)
print("Размер матрицы test:", X_test_gt50.shape)

Размер матрицы train: (95743, 50)
Размер матрицы test: (63828, 50)


In [22]:
# вывод нескольких строк
print('Векторные значения:')
X_test_gt50.head()

Векторные значения:


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,1.368033,9.440956,-2.382837,-2.361844,4.640430,-5.951034,29.204004,2.189049,4.824542,-3.292482,...,-22.780592,0.327232,4.489789,-0.203634,5.328064,-5.635204,0.529677,-1.843277,-2.686622,-2.414388
1,12.144589,11.760614,-1.919324,-1.420990,2.929482,6.436321,31.874813,-2.906626,-0.839530,1.256420,...,-31.090605,7.391714,8.837459,-1.773951,6.971064,-9.509069,4.811804,1.997747,-8.962230,-2.732993
2,3.060950,3.844096,0.097410,-0.037453,0.959599,1.422480,4.797704,0.316074,-0.663709,-1.244771,...,-7.148280,0.118720,1.647180,1.704293,1.073768,-3.173065,0.571897,1.586039,-0.344900,0.784278
3,1.636494,5.187800,-2.288114,-2.367339,-0.120624,-1.119048,13.630130,2.635803,1.327785,2.038338,...,-12.583170,-0.745174,5.736978,1.469561,4.692824,-5.189890,-1.372413,-0.900442,-0.511644,-3.045695
4,17.469137,17.929430,-2.010879,0.394288,10.650006,7.511726,56.685291,-7.435640,4.314865,4.987215,...,-43.977134,2.679926,20.633760,-5.916283,15.626647,-17.553925,7.748345,5.928867,-9.601481,-0.511386


## Шаг 2. Вывод

Для подготовки текстовых данных для дальнейшего обучения было использовано несколько техник:
- **Метод TF-IDF**. Проведена оценка важности и частоты слова во встречающемся тексте.
- **Использование Word2Vec**. Произведен расчёт векторов для каждой строки.
- **Использование сторонних корпусов**. Был использован открытый корпус *glove-twitter-50* для векторизации.

## Шаг 3. Обучение моделей

Рассмотрим, какие результаты получатся в случае обучения на выборках, подготовленных с помощью разных техник векторизации.  
В качестве моделей выберем следующие:

- [3.1. Логистическая регрессия (LogisticRegression)](#3.1.-Логистическая-регрессия-(LogisticRegression))
- [3.2. Стохастический градиентный спуск (SGDClassifier)](#3.2.-Стохастический-градиентный-спуск-(SGDClassifier))
- [3.3. Градиентный бустинг (LGBMClassifier)](#3.3.-Градиентный-бустинг-(LGBMClassifier))

В данном случае мы **не будем подбирать гиперпараметры** для моделей, а проверим все выборке на одной заранее выбранной комбинации.

### 3.1. Логистическая регрессия (LogisticRegression)

Напишем функцию, которая будет проводить проверку на тестовой выборке и возвращать **матрицу с метриками**.

In [23]:
def lr_model(X_train_name, X_test_name, method_name=None):
    """Обучение с помощью логистической регрессии для указанных выборок."""
    # обучение модели
    lr = LogisticRegression(solver='liblinear', penalty='l2', class_weight='balanced', C=15, random_state=SEED)
    lr.fit(X_train_name, y_train)
    y_pred = lr.predict(X_test_name)
    
    print(f"Значение f1 ({method_name}): {f1_score(y_test, y_pred)}")
    print()
    print(classification_report(y_test, y_pred))

In [24]:
%%time
# метод TF-IDF
lr_model(X_train_tfidf, X_test_tfidf, 'метод TF-IDF')

Значение f1 (метод TF-IDF): 0.7515021459227469

              precision    recall  f1-score   support

           0       0.98      0.96      0.97     57311
           1       0.70      0.81      0.75      6517

    accuracy                           0.95     63828
   macro avg       0.84      0.88      0.86     63828
weighted avg       0.95      0.95      0.95     63828

Wall time: 3.71 s


- На тестовой выборке значение *f1* немного **выше** необходимого **0.75**.
- Обучение происходит очень **быстро** - всего за 3-4 секунды.
- **Precision** на уровне **0.7**, значит, из 100 определенных моделью комментариев как "токсичные" реально такими является 70 комментариев.
- **Recall** на уровне **0.81**, значит, верно определяется 81% токсичных комментариев.

In [25]:
%%time
# метод Word2Vec
lr_model(X_train_vw, X_test_vw, 'метод Word2Vec')

Значение f1 (метод Word2Vec): 0.5648409282074117

              precision    recall  f1-score   support

           0       0.98      0.86      0.92     57311
           1       0.42      0.88      0.56      6517

    accuracy                           0.86     63828
   macro avg       0.70      0.87      0.74     63828
weighted avg       0.93      0.86      0.88     63828

Wall time: 14.4 s


- Результаты *f1* получились **ниже** необходимого значения **0.75**.
- **Recall** выше, чем в предыдущем случае, т. е. верно определяется 88% токсичных комментариев.
- Однако гораздо ниже **precision**, что говорит о высокой доле ложноположительных ответов.

In [26]:
%%time
# glove-twitter-50
lr_model(X_train_gt50, X_test_gt50, 'glove-twitter-50')

Значение f1 (glove-twitter-50): 0.5484558568383483

              precision    recall  f1-score   support

           0       0.98      0.86      0.92     57311
           1       0.41      0.85      0.55      6517

    accuracy                           0.86     63828
   macro avg       0.69      0.85      0.73     63828
weighted avg       0.92      0.86      0.88     63828

Wall time: 7.4 s


- Снова результат по *f1* не дотягивает до 0.75.
- Аналогичная проблема, что и в предыдущем случае - неплохое значение по **recall**, однако достаточно низкий показатель по **precision**.

### 3.2. Стохастический градиентный спуск (SGDClassifier)

In [27]:
def sgd_model(X_train_name, X_test_name, method_name=None):
    """Обучение с помощью стохастического градиентного спуска для указанных выборок."""
    # обучение модели
    sgd = SGDClassifier(loss='modified_huber', penalty='l2', class_weight='balanced', alpha=0.00001, random_state=SEED)
    sgd.fit(X_train_name, y_train)
    y_pred = sgd.predict(X_test_name)
    
    print(f"Значение f1 ({method_name}): {f1_score(y_test, y_pred)}")
    print()
    print(classification_report(y_test, y_pred))

In [28]:
%%time
# метод TF-IDF
sgd_model(X_train_tfidf, X_test_tfidf, 'метод TF-IDF')

Значение f1 (метод TF-IDF): 0.7315934065934067

              precision    recall  f1-score   support

           0       0.98      0.95      0.97     57311
           1       0.66      0.82      0.73      6517

    accuracy                           0.94     63828
   macro avg       0.82      0.88      0.85     63828
weighted avg       0.95      0.94      0.94     63828

Wall time: 356 ms


- Результат *f1* также получился немного **ниже** требуемого.

In [29]:
%%time
# метод Word2Vec
sgd_model(X_train_vw, X_test_vw, 'метод Word2Vec')

Значение f1 (метод Word2Vec): 0.31584812948000307

              precision    recall  f1-score   support

           0       0.99      0.53      0.69     57311
           1       0.19      0.96      0.32      6517

    accuracy                           0.58     63828
   macro avg       0.59      0.75      0.50     63828
weighted avg       0.91      0.58      0.66     63828

Wall time: 1.51 s


- Как и в случае с логистической регрессией, результат *f1* получился **ниже** необходимого.
- Также наблюдается сильный разрыв между значениями **precision** и **recall**.

In [30]:
%%time
# glove-twitter-50
sgd_model(X_train_gt50, X_test_gt50, 'glove-twitter-50')

Значение f1 (glove-twitter-50): 0.6303480533773559

              precision    recall  f1-score   support

           0       0.97      0.94      0.95     57311
           1       0.57      0.70      0.63      6517

    accuracy                           0.92     63828
   macro avg       0.77      0.82      0.79     63828
weighted avg       0.93      0.92      0.92     63828

Wall time: 1.2 s


- Для данной модели значение *f1* стало **выше**, однако вё равно недостаточно хорошим.
- Значительно выросо значение по **precision**, однако при этом гораздо хуже стал **recall**.

### 3.3. Градиентный бустинг (LGBMClassifier)

In [31]:
def lgbm_model(X_train_name, X_test_name, method_name=None):
    """Обучение с помощью градиентного бустинга для указанных выборок."""
    # обучение модели
    lgbm = LGBMClassifier(n_estimators=150, learning_rate=0.3, random_state=SEED)
    lgbm.fit(X_train_name, y_train)
    y_pred = lgbm.predict(X_test_name)
    
    print(f"Значение f1 ({method_name}): {f1_score(y_test, y_pred)}")
    print()
    print(classification_report(y_test, y_pred))

In [32]:
%%time
# метод TF-IDF
lgbm_model(X_train_tfidf, X_test_tfidf, 'метод TF-IDF')

Значение f1 (метод TF-IDF): 0.7671068427370948

              precision    recall  f1-score   support

           0       0.97      0.99      0.98     57311
           1       0.87      0.69      0.77      6517

    accuracy                           0.96     63828
   macro avg       0.92      0.84      0.87     63828
weighted avg       0.96      0.96      0.96     63828

Wall time: 23.8 s


In [33]:
%%time
# метод Word2Vec
lgbm_model(X_train_vw, X_test_vw, 'метод Word2Vec')

Значение f1 (метод Word2Vec): 0.64152919612464

              precision    recall  f1-score   support

           0       0.95      0.98      0.96     57311
           1       0.74      0.56      0.64      6517

    accuracy                           0.94     63828
   macro avg       0.85      0.77      0.80     63828
weighted avg       0.93      0.94      0.93     63828

Wall time: 2.37 s


In [34]:
%%time
# glove-twitter-50
lgbm_model(X_train_gt50, X_test_gt50, 'glove-twitter-50')

Значение f1 (glove-twitter-50): 0.6169056738220361

              precision    recall  f1-score   support

           0       0.95      0.98      0.96     57311
           1       0.74      0.53      0.62      6517

    accuracy                           0.93     63828
   macro avg       0.84      0.75      0.79     63828
weighted avg       0.93      0.93      0.93     63828

Wall time: 1.29 s


- Приемлемый результат получился только на выборке с *методом TF-IDF* - результат *f1* около **0.77**.

Для проверенных моделей сделаем сводную таблицу с результатами:

| Тип выборки | Способ | Результат f1 |
|:------------|:-------|-------------:|
|Метод TF-IDF|Логистическая регрессия |0.7515|
|Метод Word2Vec|Логистическая регрессия|0.5648|
|Метод glove-twitter-50|Логистическая регрессия|0.5485|
|Метод TF-IDF|Стохастический градиентный спуск |0.7316|
|Метод Word2Vec|Стохастический градиентный спуск|0.3158|
|Метод glove-twitter-50|Стохастический градиентный спуск|0.6303|
|Метод TF-IDF|Градиентный бустинг |0.7671|
|Метод Word2Vec|Градиентный бустинг |0.6415|
|Метод glove-twitter-50|Градиентный бустинг|0.6169|

## Шаг 3. Вывод

- Для большинства типов выборок наилучшие результаты получены с помощью **градиентного бустинга**.
- В нашем случае наилучшие результаты получались на выборке **TF-IDF**.
- В случае **TF-IDF** на тестовой выборке получены показатели *f1* **выше 0.75**, за исключением стохастического градиентного спуска.
- В остальных случаях показатели **не дотягивали** до нужного значения.

Стоит сделать важное замечание, что мы **не подбирали** оптимальную комбинацию гиперпараметров для каждой модели, а использовали только один вариант. Возможно, при подборе наилучших гиперпараметров мы смогли бы **улушить** результаты *f1* на других типах выборок.

# Итоговый вывод

#### Подготовка
- Полученный файл содержал **159571 комментариев**, из к-х **10%** являлись **токсичными**.
- С помощью **TextBlob Lemmatizer** проведена **лемматизация**.
- Из-за большого массива данных обработка текста была проведена **по частям (батчам)**.
- Выделены **обучающая** и **тестовая** выборки в пропорции 60:40.

Для подготовки текстовых данных для дальнейшего обучения было использовано несколько техник:
- **Метод TF-IDF**. Проведена оценка важности и частоты слова во встречающемся тексте.
- **Использование Word2Vec**. Произведен расчёт векторов для каждой строки.
- **Использование сторонних корпусов**. Был использован открытый корпус *glove-twitter-50* для векторизации.

#### Финальные результаты
| Тип выборки | Способ | Результат f1 |
|:------------|:-------|-------------:|
|Метод TF-IDF|Логистическая регрессия |0.7515|
|Метод Word2Vec|Логистическая регрессия|0.5648|
|Метод glove-twitter-50|Логистическая регрессия|0.5485|
|Метод TF-IDF|Стохастический градиентный спуск |0.7316|
|Метод Word2Vec|Стохастический градиентный спуск|0.3158|
|Метод glove-twitter-50|Стохастический градиентный спуск|0.6303|
|Метод TF-IDF|Градиентный бустинг |0.7671|
|Метод Word2Vec|Градиентный бустинг |0.6415|
|Метод glove-twitter-50|Градиентный бустинг|0.6169|

- Для большинства типов выборок наилучшие результаты получены с помощью **градиентного бустинга**.
- В нашем случае наилучшие результаты получались на выборке **TF-IDF**.
- В случае **TF-IDF** на тестовой выборке получены показатели *f1* **выше 0.75**, за исключением стохастического градиентного спуска.
- В остальных случаях показатели **не дотягивали** до нужного значения.

Стоит сделать важное замечание, что мы **не подбирали** оптимальную комбинацию гиперпараметров для каждой модели, а использовали только один вариант. Возможно, при подборе наилучших гиперпараметров мы смогли бы **улушить** результаты *f1* на других типах выборок.